# This file gives an example to do road segmentation in satellite images

## Download the image data from MASS dataset


Note: Test the code on other dataset, need to save the trainX, trainY, testX, testY image data into four file directories: '../data/xxx/yyyy/' respectively. where 'xxx' is your dataset name, 'yyyy' is 'trainX', 'trainY', 'testX', 'testY'

In [2]:
#python3.9 downloadMASS.py

## Step 1 Import and definition

In [2]:
import sys
sys.path.append('./source/') 
import source.genSamples as genS
import source.segmentor as seg
import networks as nt #User can define new network in networks.py file
import utils as ut
import time

rootDir = './'
sID = 0 #The number of test (all saved files are related to this number)
dataset = 'MASS'

sampleSize = 512 #the training sample size: sampleSize * sampleSize
alpha = '005'#0xx: 0.xx, threshold to select key pixels for preparing training sample in stage 2
ap = list(alpha); ap.insert(1,'.'); 
delta = float(''.join(ap))

network1 = 'dinknet34' #select network
lossSel1 = 'DBCE' #select loss function
modelName1 = dataset + '_' + network1 + '_' + lossSel1 # the model Name to save the training neural network

network2 = 'cunet'
lossSel2 = 'DWBCE'
modelName2 = dataset + '_' + network1 + '_' + network2 + '_' + lossSel2 + alpha

#Variables about data preparation
gtImgDict ={'MASS': '.tif', 'DeepGlobe': '.png'} #Set the format of the original images in the dataset                   
nameSplitDict = {'MASS': '.', 'DeepGlobe': '_'} #for a pair of training image and test image: before the split sign, they should have same name
nameSpliChar = nameSplitDict[dataset]     #'.' for MASS
gtPosix   =  gtImgDict[dataset]           #'.tif' for MASS

valOrTest = 'Test' #Test on the test dataset: valOrTest + 'X' and  valOrTest + 'Y'

TrainXDir = rootDir + 'data/' + dataset + '/TrainX/'      #'../data/MASS/TrainX/'
TrainYDir = rootDir + 'data/' + dataset + '/TrainY/'  #'../data/MASS/TrainY/'
testXDir  = rootDir + 'data/' + dataset + '/' + valOrTest + 'X/'
testYDir  = rootDir + 'data/' + dataset + '/' + valOrTest + 'Y/'

sampleDir1 = rootDir + 'data/' + dataset + '/TrainSamples_SC' + str(sID) + '/'  #Save the extracted training samples into this file directory
sampleDir2 = rootDir + 'data/' + dataset + '/TrainSamples_2SC' + str(sID) + '/'  #Save the extracted training samples into this file directory
sampleDir22 = rootDir + 'data/' + dataset + '/TrainSamples_2SC' + str(sID) + '_' + modelName1 + alpha + '/'

saveAsMat = False          #True: save all training samples in one numpy MAT; False: save each sample as a pair of png images
maxNum = 2000000 #Upper limit of training samples



## Step 2. Generate training samples by data augmentation

In our code, we generate the $sampleSize\times sampleSize$ training samples(default 512)

In [3]:
K = [2, 1, 1, 1, 2]  #Frequency of each random augmentation operations 
rotate = 5 
#K[0]: randomly select K[0] training samples from trainX image
#K[1], K[2]: flip trainX image from x, y axis respectively, then randomly select K[1] and K[2] training samples
#K[3]: randomly darking the trainX, then randomly select K[3] training samples
#K[4]: randomly rotate trainX image rotate times, then randomly select K[4] training samples from each rotated image
#The function generate the training samples, generated samples are saved in saveDir
genS.genTrainS4Seg(orgImgDir = TrainXDir , 
                       GTDir = TrainYDir, 
                       nameSpliChar = nameSpliChar,
                       saveDir = sampleDir1, 
                       gtPosix = gtPosix, maxNum = maxNum, K = K, rotate = rotate, sampleSize = sampleSize, 
                       saveAsMat = saveAsMat)

The estimated memory needed is: 0.0439453125
45
Total # of ORG images is: 3 [Current process on:  10078675_15 0 / 3
Current image processing time is: 6.937661170959473
Acc processing time is: 6.93766188621521
Current process on:  10078690_15 1 / 3
Current image processing time is: 6.870755195617676
Acc processing time is: 13.80853796005249
Current process on:  10078705_15 2 / 3
Current image processing time is: 7.1399688720703125
Acc processing time is: 20.94862174987793
] , Complete
(38, 512, 512, 3) (38, 512, 512, 1)


## Step 3. Define and training the nerual network in stage one

In [4]:
print('Training for Stage 1 with model ', modelName1, 'on Data ', sampleDir1) 
#Define the neural network in stage one
#Just for demo on cpu, batch_size and epochs are set to be small
net1 = nt.getNetwork(network = network1, par = nt.Parameters(inputDim = 3))
segNet = seg.segmentor(network = net1, inputDim = 3,  
                       epochs = 2, batch_size = 4, learn_rate = 0.0001, 
                       early_patience = 10, lr_patience = 5, lr_factor = 0.98, 
                       lossSel = lossSel1)
   
segNet.TrainingD(sampleDir = sampleDir1, orgDir = sampleDir1, gtDir = sampleDir1, 
                     org_pos = '_sat.npy', gt_pos = '_mask.npy',
                     ratio = 0.95, fileDir = rootDir + 'models/', fileName = modelName1, div = 2 )

Training for Stage 1 with model  MASS_dinknet34_DBCE on Data  ./data/MASS/TrainSamples_SC0/
This is DinkNet34!
The network parameters: 31.1 M
Trainable parameters: 31.1 M
Device is: cpu
Save model will be:./models/MASS_dinknet34_DBCE_segmentor.pth
the number of samples is: 41
Begining training with  DBCE  loss function
Epoch 0[--] train_losses =  0.82203 val_losses =  0.82759 val_IoU =  0.0 Time: 72.9
total training time is: 72.9
Epoch 1[--] train_losses =  0.79559 val_losses =  0.79464 val_IoU =  0.0 Time: 70.75
total training time is: 143.65


## Step 4. Test the nerual network in stage one 

In [5]:
saveDir  = rootDir + 'output/SegResult_' + dataset + '_' + modelName1 + '/'
segNet.Loading(fileName = modelName1, fileDir = rootDir + 'models/', best = 1) #best = 1 Use the network with best validation loss
ut.testGroupImages(segNet = segNet, groupDir = testXDir, groupGTDir = testYDir,
                    nameSpliChar = nameSpliChar,
                    gtImgType = gtPosix,
                    saveDir  = saveDir, sampleS = sampleSize, stepW = -1, testMax = 10000)

loading  ./models/MASS_dinknet34_DBCE_best_segmentor.pth  successfully!
0 Process image: 10828720_15
IoU =  0.0 Total number of test images is: 1
The average statitisc measures are: 
IoU =  0.0  F1 =  0.0 prec =  0.0 recall =  0.0


## Step 5. Generate training samples for stage 2

In [6]:
print('Generate 2nd Stage samples from ', sampleDir2, ' with ', modelName1, ' save in ', sampleDir22)
K = [2, 1, 1, 1, 2]      # We can set different augmentation operations
rotate = 5 
genS.genTrainS4Seg(orgImgDir = TrainXDir , 
                        GTDir = TrainYDir, 
                        nameSpliChar = nameSpliChar,
                        saveDir = sampleDir2, 
                        gtPosix = gtPosix, maxNum = maxNum, K = K, rotate = rotate, sampleSize = sampleSize, 
                        saveAsMat = saveAsMat)
    
segNet.Loading(fileName = modelName1, fileDir = rootDir + 'models/', best = 0)  #Load the stage one nerual network
genS.genRaw2ndStageTrainSamplesD(segNet, sampleDir2,  orgDir = sampleDir2, gtDir = sampleDir2, saveDir = sampleDir22, 
                                org_pos = '_sat.npy', gt_pos = '_mask.npy',
                                XRName = 'SampleX_Raw', YRName = 'SampleY_Raw',
                                netDir = rootDir + 'models/', sampleS = sampleSize, div = 20, testNum = 200000, 
                                wM = 1.0, wElse = 0.97, delta = delta, T_IoU = 0, saveAsMat = saveAsMat)


Generate 2nd Stage samples from  ./data/MASS/TrainSamples_2SC0/  with  MASS_dinknet34_DBCE  save in  ./data/MASS/TrainSamples_2SC0_MASS_dinknet34_DBCE005/
The estimated memory needed is: 0.0439453125
45
Total # of ORG images is: 3 [Current process on:  10078675_15 0 / 3
Current image processing time is: 6.5128397941589355
Acc processing time is: 6.5128419399261475
Current process on:  10078690_15 1 / 3
Current image processing time is: 6.786675930023193
Acc processing time is: 13.299637079238892
Current process on:  10078705_15 2 / 3
Current image processing time is: 6.547276020050049
Acc processing time is: 19.847028970718384
] , Complete
(41, 512, 512, 3) (41, 512, 512, 1)
loading  ./models/MASS_dinknet34_DBCE_segmentor.pth  successfully!
Total process samples: 43
Begin generating: [----------------------] Finish generating with total time 21.82
Avg IoU = 3.239422475761021e-06
Total generated train number is:  43


## Step 6. Define and training neural network in stage 2

In [7]:
print('Training for Stage 2 with model ', modelName2, 'on Data ', sampleDir22)

net2 = nt.getNetwork(network = network2, par = nt.Parameters(inputDim = 4, unet_nfilters = 32, unet_dropout = 0.08, unet_layerNum = 7))

segNet2C = seg.segmentor(network = net2, inputDim = 4,  
                       epochs = 2, batch_size = 4, learn_rate = 0.0001, 
                       early_patience = 10, lr_patience = 5, lr_factor = 0.98, 
                       lossSel = lossSel2, useMultiGPU = False)

#segNet2C.Loading(fileName = modelName2, fileDir = rootDir + 'models/', best = 0)
segNet2C.TrainingD(sampleDir = sampleDir22, orgDir = sampleDir22, gtDir = sampleDir22, 
                     org_pos = '_sat.npy', gt_pos = '_mask.npy',
                     ratio = 0.95, fileDir = rootDir + 'models/', fileName = modelName2, div = 2 )

Training for Stage 2 with model  MASS_dinknet34_cunet_DWBCE005 on Data  ./data/MASS/TrainSamples_2SC0_MASS_dinknet34_DBCE005/
This is CUNet!
UNet LayerNumer is: 7
The network parameters: 157.99 M
Trainable parameters: 157.99 M
Device is: cpu
Save model will be:./models/MASS_dinknet34_cunet_DWBCE005_segmentor.pth
the number of samples is: 43
Begining training with  DWBCE  loss function
Epoch 0[--] train_losses =  0.91194 val_losses =  0.88703 val_IoU =  0.01368 Time: 233.89
total training time is: 233.9
Epoch 1[--] train_losses =  0.86388 val_losses =  0.84216 val_IoU =  0.01424 Time: 238.58
total training time is: 472.48


## Step 7. Test the 2 stage nerual networks

In [8]:
print('Test 2 Stage segmentation')
saveDir  = rootDir + 'output/SegResult_' + dataset + '_' + modelName1 + '_' + modelName2 + '/'   
isSave = False #True, then save the test results; False: only calculate the evluation metrics
segNet.Loading(fileName = modelName1, fileDir = rootDir + 'models/', best = 0)
segNet2C.Loading(fileName = modelName2, fileDir = rootDir + 'models/', best = 1)
time_s = time.time()
ut.twoStageTestImages(segNet = segNet, segNet2 = segNet2C,
                       groupDir = testXDir, groupGTDir = testYDir,
                       nameSpliChar = nameSpliChar,
                       gtImgType = gtPosix,
                       saveDir  = saveDir, sampleS = sampleSize, stepW = -1, testMax = 10000,  T_o = 0, isSave = isSave)
    
print('Total test time is:', round(time.time() - time_s), 2)

Test 2 Stage segmentation
loading  ./models/MASS_dinknet34_DBCE_segmentor.pth  successfully!
loading  ./models/MASS_dinknet34_cunet_DWBCE005_best_segmentor.pth  successfully!
0 Process image: 10828720_15
Result for stage 1:
IoU =  0.0 Result for stage 2:
IoU =  0.0515 Avg stage1 prediction time is: 12.38 3.09
Avg stage2 prediction time is: 31.72 7.93
Avg prediction time is: 44.1 11.02
Total number of test images is: 1
The average statitisc measures for stage 1 are: 
IoU =  0.0  F1 =  0.0 prec =  0.0 recall =  0.0
The average statitisc measures for stage 2 are: 
IoU =  0.0515  F1 =  0.098 prec =  0.0515 recall =  1.0
Total test time is: 44 2
